In [1]:
pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -r requirements.txt

  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached websockets-15.0.1-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
Using cached websockets-15.0.1-cp312-cp312-win_amd64.whl (176 kB)
Using cached uvicorn-0.35.0-py3-none-any.whl (66 kB)

  Attempting uninstall: websockets

    Found existing installation: websockets 13.1

    Uninstalling websockets-13.1:

      Successfully uninstalled websockets-13.1

   ---------------------------------------- 0/2 [websockets]
   ---------------------------------------- 0/2 [websockets]
   ---------------------------------------- 0/2 [websockets]
   ---------------------------------------- 0/2 [websockets]
   ---------------------------------------- 0/2 [websockets]
  Attempting uninstall: uvicorn
   ---------------------------------------- 0/2 [websockets]
    Found existing installation: uvicorn 0.29.0
   ---------------------------------------- 0/2 [websockets]
    Uninstalling uvicorn-0.29.0:
   --------------------------

In [3]:
pip install litellm[proxy]

  Using cached uvicorn-0.29.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached websockets-13.1-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
Using cached uvicorn-0.29.0-py3-none-any.whl (60 kB)
Using cached websockets-13.1-cp312-cp312-win_amd64.whl (159 kB)

  Attempting uninstall: websockets

    Found existing installation: websockets 15.0.1

    Uninstalling websockets-15.0.1:

      Successfully uninstalled websockets-15.0.1

   ---------------------------------------- 0/2 [websockets]
   ---------------------------------------- 0/2 [websockets]
   ---------------------------------------- 0/2 [websockets]
  Attempting uninstall: uvicorn
   ---------------------------------------- 0/2 [websockets]
    Found existing installation: uvicorn 0.35.0
   ---------------------------------------- 0/2 [websockets]
    Uninstalling uvicorn-0.35.0:
   ---------------------------------------- 0/2 [websockets]
      Successfully uninstalled uvicorn-0.35.0
   ---------------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.10.0 requires uvicorn>=0.34.0, but you have uvicorn 0.29.0 which is incompatible.
google-adk 1.10.0 requires websockets>=15.0.1, but you have websockets 13.1 which is incompatible.


In [4]:
# Core dependencies
import numpy as np
import typer
import json_repair
import openai
import authlib

# Project-specific libraries
import pypdf
from tqdm import tqdm
import pandas as pd
import neo4j

import langchain
import langchain_openai
import langchain_huggingface
import langchain_community
import langchain_neo4j


# Basic Vector Search and Agents

In [5]:
#env setup
import os

from AgentRunner import AgentRunner

#get env setup
NEO4J_URI="neo4j+ssc://efea2c90.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="7V18VY7NXa1QQl06JD7_FONhdeqSap_7pUMBTgg-o3A"
NEO4J_DATABASE="neo4j"
AURA_INSTANCEID="efea2c90"
AURA_INSTANCENAME="Instance01"

from neo4j import GraphDatabase
 
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)
with GraphDatabase.driver(NEO4J_URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    


c:\Users\shant\OneDrive\Desktop\GraphRAGresume\.venv\Lib\site-packages\pydantic\_internal\_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(


In [6]:
import tiktoken
from functools import wraps

# Initialize tokenizer (adjust model name if needed)
tokenizer = tiktoken.get_encoding("o200k_base")

def count_tokens(text):
    if not text:
        return 0
    return len(tokenizer.encode(text))

# --- Patch embedding calls to count tokens on input text chunks ---
def patch_embedder_instance(embedder):
    # Patch embed_documents if exists
    if hasattr(embedder, "embed_documents"):
        original = embedder.embed_documents
        def new_embed_documents(texts, *args, **kwargs):
            for i, t in enumerate(texts):
                tokens = count_tokens(t)
                print(f"[EMBED] Document {i+1}/{len(texts)} input tokens: {tokens}")
            return original(texts, *args, **kwargs)
        embedder.embed_documents = new_embed_documents

    # Patch embed_query if exists (sometimes separate)
    if hasattr(embedder, "embed_query"):
        original = embedder.embed_query
        def new_embed_query(text, *args, **kwargs):
            tokens = count_tokens(text)
            print(f"[EMBED] Query input tokens: {tokens}")
            return original(text, *args, **kwargs)
        embedder.embed_query = new_embed_query

# --- Wrap function for inserting entities into Neo4j to count tokens ---
def insert_entities_with_token_count(driver, chunks):
    for chunk_idx, records_chunk in enumerate(chunks, start=1):
        # Count tokens in 'text' and 'id' serialized (or choose other properties)
        total_chunk_tokens = 0
        for rec_idx, record in enumerate(records_chunk, start=1):
            text = record.get("text", "")
            rec_id = record.get("id", "")
            text_tokens = count_tokens(text)
            id_tokens = count_tokens(str(rec_id))
            total = text_tokens + id_tokens
            total_chunk_tokens += total
            print(f"[NEO4J LOAD] Chunk {chunk_idx} Record {rec_idx}: text tokens = {text_tokens}, id tokens = {id_tokens}, total record tokens = {total}")

        # Run your Neo4j batch insertion (adjust to your code)
        res = driver.execute_query(
            """
            UNWIND $records AS rec
            MERGE (n:Person {id: rec.id})
            SET n.text = rec.text
            WITH n, rec
            CALL db.create.setNodeVectorProperty(n, 'embedding', rec.embedding)
            RETURN count(rec) AS records_upserted
            """,
            routing_=RoutingControl.WRITE,
            records=records_chunk,
            result_transformer_=lambda r: r.data()
        )
        print(f"[NEO4J LOAD] Chunk {chunk_idx} inserted {res[0]['records_upserted']} records with total approx {total_chunk_tokens} tokens indexed.\n")

# --- Optional: Print tokens on Neo4j vector index creation ---
def create_vector_index_with_token_logging(driver, dimension):
    print(f"[NEO4J] Creating vector index 'text_embeddings' for dimension={dimension} ...")
    driver.execute_query('''
    CREATE VECTOR INDEX text_embeddings IF NOT EXISTS FOR (n:Person) ON (n.embedding)
    OPTIONS {indexConfig: {
        `vector.dimensions`: toInteger($dimension),
        `vector.similarity_function`: 'cosine'
    }}
    ''', dimension=dimension)
    print(f"[NEO4J] Vector index creation triggered for dimension: {dimension}")

# ----- Usage example -----

# Patch your embedder instance before use:
#patch_embedder_instance(embedder)

# When you batch embed, tokens will be printed automatically for each doc chunk.

# When inserting batches into Neo4j, use the wrapped insert:
#insert_entities_with_token_count(driver, chunks(resumes_with_embeddings))

# Create vector index with print:
#create_vector_index_with_token_logging(driver, dimension=len(resumes_with_embeddings[0]["embedding"]))


## Load Documents into Neo4j

In [7]:
#read pdfs into text with unique id
from person import get_short_id
from pypdf import PdfReader
import os


#read pdfs into text with unique id
def read_resumes_from_directory(directory="C:\\Users\\shant\\OneDrive\\Desktop\\GraphRAGresume\\input"):
  
    """
    Read all PDFs from a directory and return a list of text strings
    """
    resumes = []

    # Check if directory exists
    if not os.path.exists(directory):
        print(f"Directory '{directory}' does not exist.")
        return resumes

    # Get all PDF files from the directory
    pdf_files = [f for f in os.listdir(directory) if f.lower().endswith('.pdf')]

    if not pdf_files:
        print(f"No PDF files found in '{directory}'.")
        return resumes

    # Process each PDF file
    for pdf_file in pdf_files:
        pdf_path = os.path.join(directory, pdf_file)

        try:
            # Extract text from PDF
            text = ""
            # IMPORTANT Create a unique id for each person.
            # In this case we can do it by file name but may vary by use case
            # It is important to think about how things are identified in entity extraction
            # so that they get properly resolved in the graph
            # Usually you do not want to use names.
            person_id = get_short_id(pdf_file)
            with open(pdf_path, 'rb') as file:
                pdf_reader = PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text()

            resumes.append({'id': person_id, 'text': text})
            print(f"Processed: {pdf_file} ({len(text)} characters)")
        except Exception as e:
            print(f"Error with {pdf_file}: {str(e)}")

    print(f"Total resumes loaded: {len(resumes)}")
    return resumes

resumes = read_resumes_from_directory()
resumes[0]


Processed: Abhishek_Nandgadkar.pdf (2901 characters)
Processed: ABHISHEK_PAWAR.pdf (5924 characters)
Processed: Adil_Inthiyaz_Shaik.pdf (3160 characters)
Processed: Aditi_Tyagi.pdf (13207 characters)
Processed: Aishwarya Gaba_ DataScientist 1.pdf (4581 characters)
Processed: AKSHAY_SUBRAMANYA_AITHAL.pdf (6869 characters)
Processed: ALOK_MISHRA.pdf (3632 characters)
Processed: Aman_Aslam.pdf (2771 characters)
Processed: Amit.pdf (4724 characters)
Processed: Ankith_M_G.pdf (2887 characters)
Processed: Ankush_Salunke.pdf (14065 characters)
Processed: Damodharan_Ramalingam.pdf (15131 characters)
Processed: Dashrath_Singh_Rajput.pdf (4793 characters)
Processed: Deepak_Singh.pdf (2772 characters)
Processed: DEEPAK_SONI.pdf (0 characters)
Processed: Dheeraj_B.pdf (3551 characters)
Processed: Hemant_Gautam.pdf (9648 characters)
Processed: Kshitija_Dhamdhere.pdf (2896 characters)
Processed: Kuldeep_yadav.pdf (12246 characters)
Processed: Laiba_Shaikh.pdf (4199 characters)
Processed: Mamta_Resum

{'id': 'HUc3x2rK',
 'text': 'Abhishek Nandgadkar\nData Engineer | Data Analyst | SQL | Python | Spark | Airﬂow | ADF | Big Data | Real-Time Data Processing | Kafka \n| Cloud Services (AWS, GCP , Azure) | Building High-Performance ETL Pipelines\nabhisheknandgadkar41@gmail.com 6360952485 Belgaum, Karnataka\nabhishek-nandgadkar-4760a429b\nWORK EXPERIENCE\nNubax Data Labs\nData Engineer  02/2023 - Present\n• Designed scalable ETL pipelines for large datasets, optimizing performance.\n• Built eﬃcient, reliable data workﬂows using Spark and AWS Glue, leveraging Glue jobs for schema \nmanagement and batch data processing.\n• Utilized Databricks for distributed data processing, enabling scalable, high-performance analytics and \nreducing processing times for complex data transformations.\n• Conducted data proﬁling and validation to meet data governance and quality standards.\n• Implemented CI/CD pipelines for data workﬂows, reducing deployment times and improving reliability.\n• Created and ma

In [8]:
import pandas as pd
from tqdm import tqdm
from typing import List, Dict
from neo4j_graphrag.llm import AzureOpenAILLM
from google.adk.models.lite_llm import LiteLlm
from google.adk.agents import Agent
from AgentRunner import AgentRunner  # same as you had
from google.genai.types import Part, UserContent
#add embeddings
#from langchain_openai import OpenAIEmbeddings
#from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai import AzureOpenAIEmbeddings

# embedding_model = OpenAIEmbeddings(model='text-embedding-ada-002')
#embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") # Lightweight, no API needed
embedder = AzureOpenAIEmbeddings(
        model="text-embedding-3-small",
        # This should match your Azure deployment name for the embedding model
        api_key="BmaiYil8P7o3Dgv0JzIEIA4JYd3AHl7Jh6SzBdjkwXfF4DNxCzC3JQQJ99BGACYeBjFXJ3w3AAABACOGZkhi",  # Your Azure OpenAI API key
        azure_endpoint="https://gpt-4o-intern.openai.azure.com/",
        api_version="2024-12-01-preview",
)


def chunks(xs, n=10):
    n = max(1, n)
    return [xs[i:i + n] for i in range(0, len(xs), n)]

def batch_embed_resumes(resumes:List[Dict[str,str]], embedder, chunk_size=10) -> List[Dict[str,str]]:
    df = pd.DataFrame(resumes)
    embeddings = []
    # Use tqdm to show progress during embedding generation
    for chunk in tqdm(chunks(df['text'], n=chunk_size), desc="Processing embedding chunks"):
        # Generate embeddings for each chunk and extend the embeddings list
        embeddings.extend(embedder.embed_documents(chunk))

    # combine and output
    df['embedding'] = embeddings

    #print("[Embedding] Process completed successfully.")
    return df.to_dict('records')

resumes_with_embeddings = batch_embed_resumes(resumes, embedder)
resumes_with_embeddings[0]


llm = LiteLlm(
    model="azure/gpt-4o-11-20",   # 👈 ADK expects this format
    api_base="https://raginternaffine.openai.azure.com/",
    api_key="9iApMYG4ac931NMjWX6cM0AMKhJKsC7Y6tDOPOSGAPSe7lypOGlZJQQJ99BGACMsfrFXJ3w3AAABACOGrjzw",
    api_version="2024-12-01-preview",
)

Processing embedding chunks: 100%|██████████| 4/4 [00:05<00:00,  1.31s/it]


In [9]:
import tiktoken

# Initialize tokenizer (adjust model name if needed)
tokenizer = tiktoken.get_encoding("o200k_base")

def count_tokens(text):
    if not text:
        return 0
    return len(tokenizer.encode(text))

def print_token_counts_for_nodes(records):
    """
    Calculate and print tokens used in each node's properties (like 'id', 'text')
    before inserting into Neo4j.
    
    Args:
        records: List of dicts, each dict representing a node/entity with properties.
                 Expected keys: at least 'id', 'text' (adjust if different)
    """
    total_tokens_all_nodes = 0
    for idx, record in enumerate(records, start=1):
        id_str = str(record.get("id", ""))
        text_str = record.get("text", "")
        id_tokens = count_tokens(id_str)
        text_tokens = count_tokens(text_str)
        total_node_tokens = id_tokens + text_tokens
        total_tokens_all_nodes += total_node_tokens
        print(f"[NODE {idx}] id tokens = {id_tokens}, text tokens = {text_tokens}, total tokens = {total_node_tokens}")
    print(f"[TOTAL] Tokens for all {len(records)} nodes: {total_tokens_all_nodes}")

# Example usage:
print_token_counts_for_nodes(resumes_with_embeddings)


[NODE 1] id tokens = 7, text tokens = 656, total tokens = 663
[NODE 2] id tokens = 6, text tokens = 1204, total tokens = 1210
[NODE 3] id tokens = 5, text tokens = 754, total tokens = 759
[NODE 4] id tokens = 4, text tokens = 2690, total tokens = 2694
[NODE 5] id tokens = 7, text tokens = 933, total tokens = 940
[NODE 6] id tokens = 6, text tokens = 1862, total tokens = 1868
[NODE 7] id tokens = 5, text tokens = 817, total tokens = 822
[NODE 8] id tokens = 6, text tokens = 568, total tokens = 574
[NODE 9] id tokens = 6, text tokens = 1156, total tokens = 1162
[NODE 10] id tokens = 5, text tokens = 612, total tokens = 617
[NODE 11] id tokens = 5, text tokens = 4007, total tokens = 4012
[NODE 12] id tokens = 7, text tokens = 3010, total tokens = 3017
[NODE 13] id tokens = 7, text tokens = 1086, total tokens = 1093
[NODE 14] id tokens = 5, text tokens = 565, total tokens = 570
[NODE 15] id tokens = 5, text tokens = 0, total tokens = 5
[NODE 16] id tokens = 5, text tokens = 606, total toke


### Result
![](img/graph-sample-just-docs.png)

In [10]:
from neo4j import GraphDatabase

# load into People nodes in Neo4j

#instantiate driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

#test neo4j connection
driver.execute_query("MATCH(n) RETURN count(n)")

EagerResult(records=[<Record count(n)=37>], summary=<neo4j._work.summary.ResultSummary object at 0x0000020501705BE0>, keys=['count(n)'])

In [11]:
from neo4j import RoutingControl

#create uniqueness constraint if not exists
driver.execute_query(
    'CREATE CONSTRAINT IF NOT EXISTS FOR (n:Person) REQUIRE (n.id) IS NODE KEY',
    #database_=DATABASE,
    routing_=RoutingControl.WRITE
)

#load

insert_entities_with_token_count(driver, chunks(resumes_with_embeddings))


# vector index
driver.execute_query('''
CREATE VECTOR INDEX text_embeddings IF NOT EXISTS FOR (n:Person) ON (n.embedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: toInteger($dimension),
 `vector.similarity_function`: 'cosine'
}}
''', dimension=len(resumes_with_embeddings[0]["embedding"]))

# wait for index to come online
driver.execute_query('CALL db.awaitIndex("text_embeddings", 300)')
print(f"[NEO4J] Vector index creation triggered for dimension: {len(resumes_with_embeddings[0]['embedding'])}")

[NEO4J LOAD] Chunk 1 Record 1: text tokens = 656, id tokens = 7, total record tokens = 663
[NEO4J LOAD] Chunk 1 Record 2: text tokens = 1204, id tokens = 6, total record tokens = 1210
[NEO4J LOAD] Chunk 1 Record 3: text tokens = 754, id tokens = 5, total record tokens = 759
[NEO4J LOAD] Chunk 1 Record 4: text tokens = 2690, id tokens = 4, total record tokens = 2694
[NEO4J LOAD] Chunk 1 Record 5: text tokens = 933, id tokens = 7, total record tokens = 940
[NEO4J LOAD] Chunk 1 Record 6: text tokens = 1862, id tokens = 6, total record tokens = 1868
[NEO4J LOAD] Chunk 1 Record 7: text tokens = 817, id tokens = 5, total record tokens = 822
[NEO4J LOAD] Chunk 1 Record 8: text tokens = 568, id tokens = 6, total record tokens = 574
[NEO4J LOAD] Chunk 1 Record 9: text tokens = 1156, id tokens = 6, total record tokens = 1162
[NEO4J LOAD] Chunk 1 Record 10: text tokens = 612, id tokens = 5, total record tokens = 617
[NEO4J LOAD] Chunk 1 inserted 10 records with total approx 11309 tokens indexed.


## Agent with Document Search

In [12]:
from typing import Any
from langchain_neo4j import Neo4jVector

#define tool
vector_store = Neo4jVector.from_existing_graph(
    embedder,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="text_embeddings",
    node_label="Person",
    text_node_properties=["text"],
    embedding_node_property="embedding",
)

def search_documents(search_prompt:str) -> List[Dict[str, Any]]:
    
    """
    Retrieval knowledge by searching people resumes

    """
    try:
        results = vector_store.similarity_search(search_prompt, k=5)
        return results
    except Exception as e:
        return [{"error":str(e)}]
#test
search_documents("")[:50]

c:\Users\shant\OneDrive\Desktop\GraphRAGresume\.venv\Lib\site-packages\langchain_neo4j\vectorstores\neo4j_vector.py:769: DeprecationWarning: The default returned 'id' field in the search results will be removed. Please switch to using 'elementId' instead.
  read_query, filter_params = get_search_query(


[Document(metadata={}, page_content='\ntext: '),
 Document(metadata={}, page_content="\ntext: Mr.Ankush  Salunke.  (Software  Developer)   Present  Address-Pune,   Contact  –  +91  7770076005 /  9370544177   Email  –  ankushsalunke7770@gmail.com   LinkedIn:  https://www.linkedin.com/in/ankushsalunke   \nObjective   To  work  in  a  firm  with  a  professional  work  driven  environment  where  I  can  utilize  and  apply  my   \nknowledge,\n \nskills\n \nwhich\n \nwould\n \nhelp\n \nme\n \nto\n \ngrow\n \ntechnically\n \nwhile\n \nfulfilling\n \norganizational\n \ngoals.\n  \nWith\n \nexcellent\n \ncommunication\n \nskills.\n \nResult\n \noriented,\n \nself-driven,\n \nhighly\n \nmotivated,\n \nsmart\n \nand\n  \nhungry\n \nto\n \nlearn\n \nnew\n \ntechnologies.\n \nEager\n \nto\n \ntackle\n \nnew\n \nchallenges\n \nand\n \ncollaborate\n \nwith\n \nteams\n \nto\n  \nachieve\n \nshared\n \ngoals\n \nwhile\n \nadvancing\n \norganizational\n \nsuccess.\n  \n \nProfessional  Summary   \n• 

In [ ]:
# build adk agent with Azure OpenAI + Neo4j MCP
from neo4j_graphrag.llm import AzureOpenAILLM
from google.adk.models.lite_llm import LiteLlm
from google.adk.agents import Agent
from AgentRunner import AgentRunner  # same as you had
from google.genai.types import Part, UserContent



# ✅ model parameters for consistency

#llm = LiteLlm()




In [ ]:

AGENT_INSTRUCTION = """
You are a human resources assistant who helps with exploring the resumes 
of different candidates at Shantanu's Systems.

You can access the resumes of Shantanu's System employees directly
using your `search_documents` tool. You may access this multiple
times as needed to get the information for a user.

When responding to the user:
- if your response includes people, include their names, links, roles, and any other relevant information.
- You must explain your retrieval logic and where the data came from.
- You must say exactly how relevance, similarity, etc. was inferred during search.
- You must explore all resumes documents thoroughly to find the most relevant information.
- You must respond on the basis of the retrieved information and not make assumptions.
"""

# ✅ build agent with Azure OpenAI
search_agent = Agent(
    name="search_agent",
    #model=llm,   # << replaced LiteLlm with AzureOpenAILLM instance
    description="Agent to access employee knowledge stored in documents",
    instruction=AGENT_INSTRUCTION,
    tools=[search_documents]
)

# ✅ run agent
runner = AgentRunner(app_name='search_agent', user_id='Shantanu Sharma', agent=search_agent)
await runner.start_session()


Session started successfully with ID: 1eeca815-9d9a-4993-af8f-b6f6a457ae64


True

### How many Python developers

In [15]:
import tiktoken

tokenizer = tiktoken.get_encoding("o200k_base")  # adjust if needed

async def run_with_token_count(runner, prompt_text):
    # Count prompt tokens
    prompt_tokens = len(tokenizer.encode(prompt_text))

    # Run your runner with the prompt
    result = await runner.run(prompt_text)

    # Extract answer text (adjust if result format differs)
    answer_text = result if isinstance(result, str) else getattr(result, "content", str(result))

    # Count completion tokens
    completion_tokens = len(tokenizer.encode(answer_text))

    # Print outputs with tokens info
    total_tokens = prompt_tokens + completion_tokens
    print(f"Prompt tokens: {prompt_tokens}")
    print(f"Completion tokens: {completion_tokens}")
    print(f"Total tokens: {total_tokens}\n")
    print("Answer:\n", answer_text)

    return result

# Usage
#Provide an overall summary of the talent pool in terms of skills, domains, and experience.")


In [ ]:
res = await run_with_token_count(runner, "Compare the profiles of two candidates with vector database experience.")

In [ ]:
from IPython.display import Markdown, display
display(Markdown(res))